Hello
This notebook is for my project for the  
 **Applied Data Science Capstone**  
I don't know yet what I am going to do in it.  
I am going ahead, and I will see.  

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
html_doc = 'e:List of postal codes of Canada_ M - Wikipedia.html'
with open(html_doc, "r", encoding='utf-8') as f:
     wiki_text= f.read()

In [3]:
soup = BeautifulSoup(wiki_text, 'html.parser')

find the number of rows in the postal codes table

In [4]:
table = soup.find_all('table')[0] 
num_of_rows=0
for row in table.find_all('tr'):
    num_of_rows+=1

### read the table into dataframe
in the ROW loop, we are going over all the rows in the html talbe and put it into array named data  
in the COLUMN loop, we are going over the columns: PostalCode, Borough, Neighborhood
Now, for each row, we are checking duplicated and Not assigned values and treat then


In [17]:
data= [None, None, None]
toronto_df = pd.DataFrame(columns=['PostalCode','Borough', 'Neighborhood'], index=np.arange(0, num_of_rows+1)) 
row_marker = 0
for row in table.find_all('tr'):    
    column_marker = 0
    duplicate_postal=False
    columns = row.find_all('td')           
    for column in columns:                
        data[column_marker]=column.get_text().replace( '\n', '')                                  
        column_marker+=1  
    if data[2] =='Not assigned\n':
       data[2] = data[1] 
    if data[1] != 'Not assigned':          
       #find duplicate        
       #for num_row in np.arange(0, row_marker):
           #print('in loop=', num_row, toronto_df.iat[num_row, 0], df[0])
       if toronto_df.iat[row_marker-1, 0]== data[0]:
          toronto_df.iat[row_marker-1, 2] = toronto_df.iat[row_marker-1, 2] + ', ' + data[2]
              #print('=', num_row, toronto_df.iat[num_row, 0], df[0], toronto_df.iat[num_row, 2])       
       else:
           toronto_df.iat[row_marker, 0]= data[0] 
           toronto_df.iat[row_marker, 1]= data[1] 
           toronto_df.iat[row_marker, 2]= data[2]  
           row_marker +=1
                             
toronto_df.dropna(inplace=True)
toronto_df.sort_values(by=['PostalCode'])
toronto_df

,PostalCode,Borough,Neighborhood
1,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"West Hill, Morningside, Guildwood"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea"
9,M1M,Scarborough,"Scarborough Village West, Cliffside, Cliffcrest"
10,M1N,Scarborough,"Cliffside West, Birch Cliff"


In [6]:
toronto_df.shape

(103, 3)

In [16]:
#, index_col='Postal Code'
geo_x_y = pd.read_csv('e:Geospatial_Coordinates.csv')
geo_x_y.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
toronto_df_x_y = toronto_df.merge(geo_x_y, right_on='Postal Code', left_on='PostalCode')
                                  #left_index=True, right_index=True)
toronto_df_x_y

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Morningside, Guildwood",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,M1J,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",M1K,43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea",M1L,43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West, Cliffside, Cliffcrest",M1M,43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",M1N,43.692657,-79.264848
